In [1]:
%%capture
from pathlib import Path

if Path.cwd().stem == "notebooks":
    %cd ..
    %load_ext autoreload
    %autoreload 2

In [2]:
import logging

import holoviews as hv
import hvplot.polars  # noqa
import matplotlib.pyplot as plt
import polars as pl
from polars import col

from src.data.database_manager import DatabaseManager
from src.log_config import configure_logging
from src.plots.utils import prepare_multiline_hvplot

configure_logging(stream_level=logging.DEBUG, ignore_libs=("Comm", "bokeh", "tornado"))
logger = logging.getLogger(__name__.rsplit(".", maxsplit=1)[-1])


hv.output(widget_location="bottom", size=130)

In [3]:
db = DatabaseManager()

In [4]:
with db:
    stimulus = db.get_table("Feature_Stimulus")
    labels = db.get_table("Labels")

In [5]:
df = labels.filter(col("major_decreasing_intervals") > 0)

In [6]:
df

trial_id,trial_number,participant_id,rownumber,timestamp,temperature,rating,stimulus_seed,skin_area,normalized_timestamp,decreasing_intervals,major_decreasing_intervals,increasing_intervals,plateau_intervals,prolonged_minima_intervals
u16,u8,u8,u32,f64,f64,f64,u16,u8,f64,i64,i64,i64,i64,i64
1,1,1,690,363232.998,0.967948,1.0,396,1,69008.667,2,1,0,0,0
1,1,1,691,363331.7346,0.96789,1.0,396,1,69107.4036,2,1,0,0,0
1,1,1,692,363432.4644,0.967715,1.0,396,1,69208.1334,2,1,0,0,0
1,1,1,693,363534.196,0.967423,1.0,396,1,69309.865,2,1,0,0,0
1,1,1,694,363633.926,0.967015,1.0,396,1,69409.595,2,1,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
332,12,28,21477,2.7637e6,0.073359,0.6,133,1,166587.1057,1659,996,0,0,0
332,12,28,21478,2.7638e6,0.07307,0.6,133,1,166686.839,1659,996,0,0,0
332,12,28,21479,2.7639e6,0.072897,0.6,133,1,166787.5704,1659,996,0,0,0


In [7]:
# Normalize time for major decreasing intervals
df = df.with_columns(
    (
        col("timestamp") - col("timestamp").min().over("major_decreasing_intervals")
    ).alias("normalized_time")
)

In [8]:
df = prepare_multiline_hvplot(
    df,
    time_column="normalized_time",
    trial_column="major_decreasing_intervals",
)
df

trial_id,trial_number,participant_id,rownumber,timestamp,temperature,rating,stimulus_seed,skin_area,normalized_timestamp,decreasing_intervals,major_decreasing_intervals,increasing_intervals,plateau_intervals,prolonged_minima_intervals,normalized_time
u16,u8,u8,f64,f64,f64,f64,u16,f64,f64,i64,i64,i64,i64,i64,f64
1,1,1,690.0,363232.998,0.967948,1.0,396,1.0,69008.667,2,1,0,0,0,0.0
1,1,1,691.0,363331.7346,0.96789,1.0,396,1.0,69107.4036,2,1,0,0,0,98.7366
1,1,1,692.0,363432.4644,0.967715,1.0,396,1.0,69208.1334,2,1,0,0,0,199.4664
1,1,1,693.0,363534.196,0.967423,1.0,396,1.0,69309.865,2,1,0,0,0,301.198
1,1,1,694.0,363633.926,0.967015,1.0,396,1.0,69409.595,2,1,0,0,0,400.928
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
332,12,28,21478.0,2.7638e6,0.07307,0.6,133,1.0,166686.839,1659,996,0,0,0,19682.1372
332,12,28,21479.0,2.7639e6,0.072897,0.6,133,1.0,166787.5704,1659,996,0,0,0,19782.8686
332,12,28,21480.0,2.7640e6,0.072839,0.6,133,1.0,166887.304,1659,996,0,0,0,19882.6022


In [15]:
df.hvplot(
    x="normalized_time",
    y=["rating"],
    groupby="participant_id",
    kind="step",
    width=800,
    height=400,
    widget_type="scrubber",
    widget_location="bottom",
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'b5253cad-82e0-4248-ae26-9587806d2af5': {'version…

In [65]:
# Add a column for the interval in the trial to be able to group by it
# That way we can dicriminate between the different major decreasing intervals in one
# stimulus seed
df = df.with_columns(
    ((col("major_decreasing_intervals") - 1) % 3 + 1).alias(
        "major_decreasing_interval_in_trial"
    )
)

In [66]:
df.hvplot(
    x="normalized_time",
    y=["rating"],
    groupby=["stimulus_seed", "major_decreasing_interval_in_trial"],
    kind="step",
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'c3babce6-d636-4bff-a143-1c10bc72ee5d': {'version…

In [68]:
# filter all intervals out that do not cover the whole rating spectrum ranging from 0 to 100
agg_ratings = df.group_by("major_decreasing_intervals", maintain_order=True).agg(
    min_rating=col("rating").min(),
    max_rating=col("rating").max(),
)
agg_ratings

agg_ratings = agg_ratings.filter(col("min_rating") == 0).filter(col("max_rating") == 1)
agg_ratings

major_decreasing_intervals,min_rating,max_rating
i64,f64,f64
1,0.0,1.0
2,0.0,1.0
4,0.0,1.0
5,0.0,1.0
6,0.0,1.0
…,…,…
924,0.0,1.0
955,0.0,1.0
975,0.0,1.0
